# TextRank notebook for reproducibility

This notebook contains a full walkthrough on how to use the code on the Reddit TIFU dataset. The output is done on a 30 post subset.

## Library imports

In [1]:
## Data structures
import pandas as pd # Data/tabular structures
import numpy as np # Linear algebra

## Natural language processing
import nltk # Natural language toolkit
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
# nltk.download('punkt') # one time execution

## Utilities

import re # RegEx
import math 
from tqdm import tqdm # Progress bars

## Similarity calculation

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx # Graph construction

## Evaluation notebook

from rouge import Rouge


## Function Initializations

Shoutout to the author of: https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/ for inspiration and example of a TextRank implementation.

In [2]:
# used to import Glove embeddings

def create_word_embeddings(embedding_path):

    word_embedding_dict = {} # Create return object
    
    f = open(embedding_path, encoding='utf-8') # Open embedding file

    # Process file
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embedding_dict[word] = coefs
    f.close()
    
    print("Word embeddings succesfully extracted!")
    
    return(word_embedding_dict)

In [3]:
# Used to preprocess data

def remove_stopwords(sen): # Remove stopwords using NLTK stopwords
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

def pre_process_single_doc(doc):
    
    sentences = [] # Create empty temporary list
    
    for s in doc['documents']:
        sentences.append(sent_tokenize(s))
    
    sentences = [y for x in sentences for y in x] # flatten list
    
    sentences = list(pd.Series(sentences)) # Transform to list
    
    # Replace HTML/Markup

    clean_sentences = list(map(lambda x: x.replace('\n','').replace('"b','').replace('\\n','').replace("'b",'').replace('b"','').replace("b'",''),sentences))
    
    # Replace URL's

    clean_sentences = [re.sub('(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])','URL',i) for i in clean_sentences]
    
    # Replace subreddit indicator
    
    clean_sentences = list(map(lambda x: x.replace('r/','subreddit '),clean_sentences))
    
    # Remove punctuation and special characters

    clean_sentences = [re.sub(r"[^a-zA-Z0-9]+"," ",i) for i in clean_sentences]

    # Lowercase everything 

    clean_sentences = [s.lower() for s in clean_sentences]

    # remove stopwords from the sentences

    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    
    return(sentences, clean_sentences) # Return preprocessed sentences

In [4]:
# Used to transform sentence object to a word embedding vector

def create_sentence_vectors(preprocessed_doc):

    sentence_vectors = []
    
    # Use embedding document to create embedding vector
    for i in preprocessed_doc:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)
    
    return(sentence_vectors) # Return the embedding vector

In [5]:
# Get longest common substring 

def calculate_lcs(string1, string2):
    lcs = 0;
    
    # Calculate the LCS using two strings (sentences)
    for a in range(len(string1)):
         for b in range(len(string2)):
            k = 0;
            while ((a + k) < len(string1) and (b + k) < len(string2)
        and string1[a + k] == string2[b + k]):
                k = k + 1;

            lcs = max(lcs, k);
    return lcs; # Return LCS

In [6]:
# Create Similarity Matrix based on Cosine Similarity

def create_similarity_matrix_cosine_vector(sen_vec,sen_list):

    # Create similarity matrix

    sim_mat = np.zeros([len(sen_list), len(sen_list)]) # Initialize similarity matrix

    for i in range(len(sen_list)): # Pair wise similarity calculation
        for j in range(len(sen_list)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sen_vec[i].reshape(1,100), sen_vec[j].reshape(1,100))[0,0] # Append to matrix
    
    return(sim_mat) # Return matrix object

In [7]:
# Create Similarity Matrix for TF-IDF

def create_similarity_matrix_TFIDF(sen_list):

    # Create similarity matrix

    vect = TfidfVectorizer(min_df=1, stop_words="english") # Use library for pairwise TFIDF calculation
    tfidf = vect.fit_transform(sen_list)     
    pairwise_similarity = tfidf * tfidf.T # Create matrix
    sim_mat = pairwise_similarity.toarray() 
    np.fill_diagonal(sim_mat, 0) # Zero out self similarity

    return(sim_mat) # Return matrix object

In [8]:
# Create Similarity Matrix based on LCS Similarity

def create_similarity_matrix_lcs(sen_list):

    # Create similarity matrix

    sim_mat = np.zeros([len(sen_list), len(sen_list)]) # Initialize empty matrix

    for i in range(len(sen_list)): # Pairwise
        for j in range(len(sen_list)):
            if i != j:
                sim_mat[i][j] = calculate_lcs(sen_list[i], sen_list[j]) # Append LCS
    
    return(sim_mat) # Return matrix object

In [9]:
# Generate Graph from Similarity Matrix

def create_graph_and_rank(sim_mat,sen_list,plot_graph=False):
    
    # Create graph

    nx_graph = nx.from_numpy_array(sim_mat)

    # Call Pagerank

    scores = nx.pagerank_numpy(nx_graph)
    
    if plot_graph == True:
        # Plot the graph
        nx.draw(nx_graph)
    
    # Rank Sentences Based on PageRank Algorithm
    
    return(sorted(((scores[i],s) for i,s in enumerate(sen_list)), reverse=True)) # Return ranking

In [10]:
# Extract top sentences as the summary

def generate_summary(sen_ranks, sen_list):

    # Use 20% of the length of the original post

    sum_size = math.ceil(int(0.2 * len(sen_list)))
    
    if sum_size < 1: # Fail-safe
        sum_size = 1

    order_dict = dict()
    
    # Extract Top-N and re-order
    
    for i in range(sum_size):
        order_dict.update({sen_ranks[i][1]: sen_list.index(sen_ranks[i][1])})

    sorted_values = sorted(order_dict.values()) # Sort the values
    sorted_dict = {}

    for i in sorted_values:
        for k in order_dict.keys():
            if order_dict[k] == i:
                sorted_dict[k] = order_dict[k]
                break
                
    # Conduct some pre-processing on output to increase readibility

    clean_sentences = list(map(lambda x: x.replace('\n','').replace('"b','').replace('\\n','').replace("'b",'').replace('b"','').replace("b'",''),sorted_dict))

    clean_sentences = [re.sub('(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])','URL',i) for i in clean_sentences]

    clean_sentences = list(map(lambda x: x.replace('r/','subreddit '),clean_sentences))

    clean_sentences = [re.sub(r"[^a-zA-Z0-9%',.-]+"," ",i) for i in clean_sentences]
    
    return(" ".join(clean_sentences)) # Return summary

In [11]:
# Conduct some pre-processing on the source data to remove noise

def clean_document(column):
    column_new = list(map(lambda x: x.replace('\n','').replace('"b','').replace('\\n','').replace("'b",'').replace('b"','').replace("b'",''),column))
    
    column_new = [re.sub('(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])','URL',i) for i in column_new]
    
    column_new = [re.sub(r"[^a-zA-Z0-9%!?:;,.-]+"," ",i) for i in column_new]
                                                                                                                   
    column_output = list(map(lambda x: x.replace('r/','subreddit '),column_new))

    return(column_output) # Return processed data

## Reading in the sample data

In [12]:
tifu_dataset = pd.read_csv("tifu-dataset-snippet.csv") # Example in the GitHub

In [13]:
tifu_dataset.info() # Inspect the columns/ shape (30 subset)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         30 non-null     int64  
 1   documents     30 non-null     object 
 2   num_comments  30 non-null     int64  
 3   score         30 non-null     int64  
 4   title         30 non-null     object 
 5   tldr          30 non-null     object 
 6   ups           30 non-null     int64  
 7   upvote_ratio  30 non-null     float64
dtypes: float64(1), int64(4), object(3)
memory usage: 2.0+ KB


In [14]:
tifu_dataset.head() # Visually inspect the data

,index,documents,num_comments,score,title,tldr,ups,upvote_ratio
0,5900,"b""this happened to me yesterday and i'm still ...",2715,85249,b'having my reddit history revealed by jimmy k...,"b""jimmy kimmel had reddit co-founder alexis oh...",85249,0.88
1,34594,b'edit: i got an ama thread now. help me: \n\n...,3825,81547,b'cumming into a coconut',"b""don't fuck coconuts.""",81547,0.90
2,37412,"b'recently, i traveled to denver, colorado wit...",4262,80916,"b""stuffing my face with edibles before dinner ...",b'ate way too many edibles on a trip and wigge...,80916,0.88
3,37333,b'this happened sunday night.\n\n\n\nmy oldest...,1166,49877,b'not telling my wife our son was coming home',"b""didn't tell wife that marine son was coming ...",49877,0.87
4,4506,b'i\'m in my third year of university taking e...,2466,48085,b'sitting in the wrong class for an entire mon...,b'i was in the wrong econ class for an entire ...,48085,0.90


## Running the Pipeline

In [15]:
# Read in word embeddings (downloadable at https://nlp.stanford.edu/projects/glove/)

word_embeddings = create_word_embeddings('glove.6b/glove.6B.100d.txt')

stop_words = stopwords.words('english')

Word embeddings succesfully extracted!


In [16]:
# Execute the TextRank algorithm (refer to the pdf in the github for a written explanation)

def pipeline_textrank_summarization(doc,sim_measure): # Sim measure should be either cosine, lcs or TFIDF

    if sim_measure == "cosine":
        
        sentence_list, pre_processed_sentences = pre_process_single_doc(doc)

        sentence_vector = create_sentence_vectors(pre_processed_sentences)

        similarity_matrix = create_similarity_matrix_cosine_vector(sentence_vector,sentence_list)

        ranking_scores = create_graph_and_rank(similarity_matrix,sentence_list)

        summary = generate_summary(ranking_scores,sentence_list)
                
    if sim_measure == "lcs":
        
        sentence_list, pre_processed_sentences = pre_process_single_doc(doc)
        
        similarity_matrix = create_similarity_matrix_lcs(pre_processed_sentences)

        ranking_scores = create_graph_and_rank(similarity_matrix,sentence_list)

        summary = generate_summary(ranking_scores,sentence_list)
        
    if sim_measure == "tfidf":
        
        sentence_list, pre_processed_sentences = pre_process_single_doc(doc)
        
        similarity_matrix = create_similarity_matrix_TFIDF(pre_processed_sentences)

        ranking_scores = create_graph_and_rank(similarity_matrix,sentence_list)

        summary = generate_summary(ranking_scores,sentence_list)
                
    return(summary)

## One post example

In [17]:
sample_post = tifu_dataset[8:9] # Select one document

# Print post and TLDR

text = sample_post['documents'][8]
print("_____________TEXT______________")
print(text)

tldr = sample_post['tldr'][8]
print("_____________TLDR______________")
print(tldr)

_____________TEXT______________
b'this happened only minutes ago.\n\n\nthe graphics card in my old ps2 decided it wanted to give up on me recently, so i decided to replace it when i had a little extra cash. i was out browsing different sites like craigslist and the like, when i stumbled upon the ps2 mentioned in the title. it looked like a great deal at the time. $25 to buy it from this guy, whereas a secondhand store in town was selling them for around $45 to $60.\n\n\nat the time, this seemed like a no-brainer.\n\n\nnow, i should preface this by saying that i have a strange faith in the honesty of others. benefit of the doubt and all that noise. after all, the car i drive now is one i bought from a guy on the internet, and it runs great for something that is 27 years old. why should this be any different?\n\n\nstarting to sound like a mistake yet?\n\n\nif the answer is "no," then have no fear. that is almost certainly about to change. the model the seller advertised on letgo was one 

In [18]:
# Generate and print 3 summaries of the sample

print("_______ LCS method _______")

summary_lcs = pipeline_textrank_summarization(sample_post,"lcs")
print(summary_lcs)

print("_______ cosine method _______")

summary_cosine = pipeline_textrank_summarization(sample_post,"cosine")
print(summary_cosine)

print("_______ TFIDF method _______")

summary_tfidf = pipeline_textrank_summarization(sample_post,"tfidf")
print(summary_tfidf)

_______ LCS method _______
 25 to buy it from this guy, whereas a secondhand store in town was selling them for around 45 to 60.at the time, this seemed like a no-brainer.now, i should preface this by saying that i have a strange faith in the honesty of others. he told me he had already sold the larger one.my first instinct, as many logical redditors would tell me, is that i should have walked away when i saw i was being sold something that was improperly advertised. i just wanted to get this thing and go home.back at the ole ranch, i hooked up the console, slapped in kingdom hearts ii, and got ready to enjoy the rest of my day. i repeated this several times with discs i knew would work same story.now i 'm starting to get slightly pissed. i 'm sure you can see where this next part is going.i open up the app to message the seller and let him know he had sold me a defective console. after a video about a quick mod i could make to the system, i was feeling pretty confident that i was goin

In [19]:
# Calculate ROUGE scores for sample

rouge = Rouge()

rouge.get_scores(summary_tfidf, tldr)

[{'rouge-1': {'r': 0.5, 'p': 0.05555555555555555, 'f': 0.09999999820000001},
  'rouge-2': {'r': 0.058823529411764705,
   'p': 0.0038910505836575876,
   'f': 0.007299268909105626},
  'rouge-l': {'r': 0.5, 'p': 0.05555555555555555, 'f': 0.09999999820000001}}]

## Full dataset summarization

The summarization of the sample should be relatively quick, for reference the full dataset took ~13 hours. Your experience may vary on hardware setup.

In [20]:
# Lists to save summaries

summary_list_LCS = []
summary_list_Cosine = []
summary_list_TFIDF = []

for index, row in tqdm(tifu_dataset.iterrows(), total=tifu_dataset.shape[0]): # Loop over entire collection
    curr_post = tifu_dataset[index:index+1] # Select one document
    
    try: # Catching errors so that the process can't fail midway one one sample
        summary_list_Cosine.append(pipeline_textrank_summarization(curr_post,"cosine")) # Glove cosine similarity
    except Exception:
        summary_list_Cosine.append("error")
        
    try:
        summary_list_LCS.append(pipeline_textrank_summarization(curr_post,"lcs")) # LCS
    except Exception:
        summary_list_LCS.append("error")
                
    try:
        summary_list_TFIDF.append(pipeline_textrank_summarization(curr_post,"tfidf")) # TF-IDF
    except Exception:
        summary_list_TFIDF.append("error")

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [01:16<00:00,  2.56s/it]


In [21]:
# Extract reference TLDR as a list

reference_list = list(tifu_dataset['tldr'])

In [22]:
# Append new summaries to the dataframe

tifu_dataset["LCS"] = summary_list_LCS
tifu_dataset["COSINE"] = summary_list_Cosine
tifu_dataset["TFIDF"] = summary_list_TFIDF

# Clean a few columns of noise

documents = tifu_dataset.documents
title = tifu_dataset.title
tldr = tifu_dataset.tldr

tifu_dataset["documents"] = clean_document(documents)
tifu_dataset["title"] = clean_document(title)
tifu_dataset["tldr"] = clean_document(tldr)

In [23]:
tifu_dataset.head() # New dataframe

,index,documents,num_comments,score,title,tldr,ups,upvote_ratio,LCS,COSINE,TFIDF
0,5900,this happened to me yesterday and i m still ba...,2715,85249,having my reddit history revealed by jimmy kim...,jimmy kimmel had reddit co-founder alexis ohan...,85249,0.88,this alone was crazy for me because i never ex...,however it ended up on the front page with muc...,i was mind blown over this.they proceeded talk...
1,34594,edit: i got an ama thread now. help me: URL ti...,3825,81547,cumming into a coconut,don t fuck coconuts.,81547,0.90,i end up grabbing the coconut drill and throug...,one day i hear that my mother is going to be o...,edit i got an ama thread now. horny me decides...
2,37412,"recently, i traveled to denver, colorado with ...",4262,80916,stuffing my face with edibles before dinner wi...,ate way too many edibles on a trip and wigged ...,80916,0.88,what could possibly go wrong so the first thin...,"as a result, before leaving, i begged my wife ...","recently, i traveled to denver, colorado with ..."
3,37333,this happened sunday night.my oldest son is in...,1166,49877,not telling my wife our son was coming home,didn t tell wife that marine son was coming ho...,49877,0.87,"i get to the airport, and actually watched the...",now the whole time ive been texting my wife sa...,now the whole time ive been texting my wife sa...
4,4506,i m in my third year of university taking engi...,2466,48085,sitting in the wrong class for an entire month...,i was in the wrong econ class for an entire mo...,48085,0.90,i 'm in my third year of university taking eng...,i 'm in my third year of university taking eng...,"i came to class. so the class before the exam,..."


## Evaluation

This is normally done in a seperate notebook but for the sake of having all sources at one place here it is demonstrated.

In [24]:
rouge.get_scores(summary_list_LCS, reference_list,avg=True) # LCS

{'rouge-1': {'r': 0.355054805272828,
  'p': 0.07393664754696957,
  'f': 0.11237401721113938},
 'rouge-2': {'r': 0.07395258537429861,
  'p': 0.009412406316520816,
  'f': 0.015862560120891957},
 'rouge-l': {'r': 0.3116353387853391,
  'p': 0.06040301101219256,
  'f': 0.09446451371731314}}

In [25]:
rouge.get_scores(summary_list_Cosine, reference_list,avg=True) # Glove cosine

{'rouge-1': {'r': 0.3373625388625309,
  'p': 0.07447375794115715,
  'f': 0.11375380781620383},
 'rouge-2': {'r': 0.07458305457942253,
  'p': 0.013203658763387869,
  'f': 0.02056554092145283},
 'rouge-l': {'r': 0.3074820343460368,
  'p': 0.06778743921906497,
  'f': 0.10340511227595182}}

In [26]:
rouge.get_scores(summary_list_TFIDF, reference_list,avg=True) # TF-IDF

{'rouge-1': {'r': 0.3528981329028243,
  'p': 0.08357784203772847,
  'f': 0.12763933018016813},
 'rouge-2': {'r': 0.06966936364793404,
  'p': 0.010628152375986993,
  'f': 0.017523173576019103},
 'rouge-l': {'r': 0.31325314458319087,
  'p': 0.073493102461619,
  'f': 0.11282576019471463}}

## Export new data

In [27]:
# Create compression to do it in a fast fashion
compression_opts = dict(method='zip', archive_name='TextRankOutputSample.csv')  # Create a compression method to efficiently export data
tifu_dataset.to_csv('TextRankOutputSample.zip', index=True, compression=compression_opts) # Export the data